<a href="https://colab.research.google.com/github/IgorBaratta/FEniCSxCourse/blob/ICMC23/Problem5_Navier-Stokes/navier_stokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Incompressible Navier-Stokes equations

## Strong formulation

The plan for this tutorial is to solve the prototypical **non-linear mixed** problem in fluid mechanics, namely, the **Navier-Stokes** equations in the incompressible regime.
The main difficulties when discretizing the Navier-Stokes problem are related to
the treatment of:

* the pressure-velocity coupling (there is no equation for $p$;
* the convective nonlinear term $\rho \,(\mathbf{u}\cdot \nabla)~\mathbf{u}$
* boundary layers and turbulent flows

Let us recall the problem in strong form. Given an initial velocity field $\mathbf{u}_0$ that satisfies the incompressibility constraint ($\nabla \cdot \mathbf{u}_0 = 0$), we aim to find the velocity $\mathbf{u}$ and pressure $p$ such that

\begin{equation}
\left\{
\begin{array}{ll}
\rho\left (\dfrac{\partial{\mathbf{u}}}{\partial{t}} +  \mathbf{u} \cdot \nabla \mathbf{u}\right )
- \nabla \cdot ( 2 \, \mu \boldsymbol{\varepsilon}(\mathbf{u})) + \nabla{p} = \mathbf{f} & \mbox{ in } \Omega \\
& \\
\nabla \cdot \mathbf{u} = 0  & \mbox{ in } \Omega \\
& \\
\mathbf{u} = \mathbf{u}_D & \mbox{ on } \Gamma_D \\
& \\
\left [-p\mathbf{I} + 2\,\mu\, \boldsymbol{\varepsilon}(\mathbf{u}) \right ] \cdot \check{\mathbf{n}} = \boldsymbol{\mathcal{F}} & \mbox{ on } \Gamma_N
\end{array}
\right.
\end{equation}
$~$

where $\mathbf{u}_D$ is a given function on $\Gamma_D$, $\boldsymbol{\mathcal{F}}$ is a given function on $\Gamma_N$ and $\mathbf{f}$ are the body forces (e.g., $\mathbf{f} = -\rho \, g  \check{\mathbf{e}}_3$).

The following comments are in order:

* The stresses in the fluid are described by the Cauchy stress tensor

\begin{equation}
\boldsymbol{\sigma}(\mathbf{x},t) = -p(\mathbf{x},t) \mathbf{I} + \boldsymbol{\sigma}^*(\mathbf{x},t)
\end{equation}

which is the sum of a volumetric part 

\begin{equation}
-p(\mathbf{x},t) \mathbf{I} = 
\begin{bmatrix}
-p & 0 & 0 \\
0 & -p & 0 \\
0 & 0 & -p
\end{bmatrix}
\end{equation}
and a deviatoric part
\begin{equation}
\boldsymbol{\sigma}^*(\mathbf{x},t)= 2 \, \mu \boldsymbol{\varepsilon}(\mathbf{u}) =
\mu \left (\nabla{\mathbf{u}} + \nabla^{\intercal}{\mathbf{u}} \right)
\end{equation}

where $\mu$ is the dynamic viscosity of the fluid. 

* The pressure $p$ has no thermodynamic meaning for this problem, its 
sole role is to enforce the incompressibility constraint

$$
\nabla \cdot \mathbf{u} = 0
$$


## Solution strategy and variational formulation

There are essentially two approaches to solve this problem, namely,

* **Segregated**: typically, some variant of the Chorin's projection method, in which the problem is splitted into a 
    * Momentum predictor to find a velocity $\mathbf{u}^*$ ignoring the pressure term
    * Poisson's pressure solver to find $p$
    * Momentum corrector to find a velocity $\mathbf{u}$ that satisfies the incompressibility constraint

* **Monolithic**: both fields $\mathbf{u}$ and $p$ are found at once, leading
to an algebraic problem with saddle point structure

In this tutorial, we will adopt the **monolithic** approach for simplicity and also to continue illustrating the use mixed function spaces. This approach is certainly the most appropriate for low Reynolds (Re $\ll 1$) numbers, wheareas the segregated approach is more convenient for Re $\gg 1$.

The variational formulation for this problem shares similarities with that of the Elasticity problem, however, we have additional terms to deal with.
By multiplying by a sufficiently regular test function $\mathbf{v}$ belonging to the velocity space in the first equation and integrate by parts
and, multiplying by a test function $q$ belonging to the pressure space
in the second equation we obtain

Find $(\mathbf{u},p) \in U_{\mathbf{g}} \times L^2(\Omega)$ such that

\begin{eqnarray}
\int_{\Omega}{\mathcal{G}_{\mathbf{u}} \cdot \mathbf{v} }\,dx +
{\int_{\Omega}}{2\,\mu \boldsymbol{\varepsilon}(\mathbf{u}) : \boldsymbol{\varepsilon}(\mathbf{v}) }\,dx 
 - {\int_{\Omega}}{p\,\nabla \cdot \mathbf{v}}\,dx & = & {\int_{\Omega}}{\mathbf{f}\cdot \mathbf{v}}\,dx + {\int_{\Gamma_{\boldsymbol{\mathcal{F}}}}}{\boldsymbol{\mathcal{F}} \cdot \mathbf{v}}\,ds \\
& &  \\
{\int_{\Omega}}{q\,\nabla \cdot \mathbf{u}}\,dx & = &  0 
\end{eqnarray}
$\forall (\mathbf{v},q) \in U_{\mathbf{0}} \times L^2(\Omega).$

where $\mathcal{G}_{\mathbf{u}}  = \rho\left ({\partial_t{\mathbf{u}}} +  \mathbf{u} \cdot \nabla \mathbf{u}\right )~$ is the inertial term.
$~$

In abstract form:

\begin{equation}
\left \{
\begin{array}{rlll}
c(\mathbf{u};\mathbf{u}, \mathbf{v}) + {a}(\mathbf{u},\mathbf{v}) - b(\mathbf{v},p) & = & \ell(\mathbf{v}) & \forall \mathbf{v} \in U_{\mathbf{0}} \nonumber \\
b(\mathbf{u},q) & = & 0 & \forall q\in L^2(\Omega) \nonumber
\end{array}
\right.
\end{equation}

where the form $c(\cdot ; \cdot, )$ corresponds to the inertial terms, $a(\cdot, \cdot)$ to the viscous terms and $b(\cdot, \cdot)$ to the pressure and continuity terms. 
$~$

Adding everything as usual for the `FEniCSx` implementation
$~$

---
Find $(\mathbf{u}, p) \in W = U_{\mathbf{g}} \times Q$ such that 

$$
B\left ( (\mathbf{u}, p),  (\mathbf{v}, q) \right) = L((\mathbf{v}, q))
$$

$\forall (\mathbf{v}, q) \in U_{\mathbf{0}} \times Q$ with

---

## Finite element implementation

In [ ]:
try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh
    
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

try:
    import pyvista
except ImportError:
    !pip install - q piglet pyvirtualdisplay ipyvtklink pyvista panel
    !apt-get - qq install xvfb
    import pyvista

In [ ]:
from dolfinx import mesh, fem, io, plot, nls, log, cpp

from ufl import (derivative, SpatialCoordinate, FiniteElement, VectorElement, MixedElement, 
                 TestFunction, split, Identity, Measure, dx, ds, 
                 grad, nabla_grad, div, dot, inner, as_vector, FacetNormal)

import numpy as np
from mpi4py import MPI
from petsc4py.PETSc import ScalarType, Options
import matplotlib.pyplot as plt
import IPython


We will solve two problems in this tutorial:

* The simple Poiseuille channel flow

* The Flow around a circular obstacle

The geometry for each case is created as in other tutorial we have done.
Being a complex geometry, the second case requires a unstructured grid
which take some additional programming efforts. 
$~$

You have to select the case by setting the `simple_mesh` boolean variable
below (`True` for the simple channel flow and `False` otherwise):


In [ ]:
# Computational mesh 
simple_mesh = False

if(simple_mesh):
  Lx, Ly = 3.0, 1.0
  msh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                              points=((0.0, 0.0), (Lx, Ly)), n=(48, 16),
                              cell_type=mesh.CellType.triangle)
else:
  gmsh.initialize()
  L = 1.75
  H = 0.41
  Lx, Ly = L, H
  c_x = c_y =0.2
  r = 0.05
  gdim = 2
  mesh_comm = MPI.COMM_WORLD
  model_rank = 0
  
  fluid_marker = 1
  inlet_marker, outlet_marker, wall_marker, obstacle_marker = 2, 3, 4, 5

  if mesh_comm.rank == model_rank:
    rectangle = gmsh.model.occ.addRectangle(0,0,0, L, H, tag=1)
    obstacle = gmsh.model.occ.addDisk(c_x, c_y, 0, r, r)

    fluid = gmsh.model.occ.cut([(gdim, rectangle)], [(gdim, obstacle)])
    gmsh.model.occ.synchronize()

    volumes = gmsh.model.getEntities(dim=gdim)
    assert(len(volumes) == 1)
    gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], fluid_marker)
    gmsh.model.setPhysicalName(volumes[0][0], fluid_marker, "Fluid")

  inflow, outflow, walls, obstacle = [], [], [], []
  if mesh_comm.rank == model_rank:
    boundaries = gmsh.model.getBoundary(volumes, oriented=False)
    for boundary in boundaries:
        center_of_mass = gmsh.model.occ.getCenterOfMass(boundary[0], boundary[1])
        if np.allclose(center_of_mass, [0, H/2, 0]):
            inflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L, H/2, 0]):
            outflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L/2, H, 0]) or np.allclose(center_of_mass, [L/2, 0, 0]):
            walls.append(boundary[1])
        else:
            obstacle.append(boundary[1])

    gmsh.model.addPhysicalGroup(1, walls, wall_marker)
    gmsh.model.setPhysicalName(1, wall_marker, "Walls")
    gmsh.model.addPhysicalGroup(1, inflow, inlet_marker)
    gmsh.model.setPhysicalName(1, inlet_marker, "Inlet")
    gmsh.model.addPhysicalGroup(1, outflow, outlet_marker)
    gmsh.model.setPhysicalName(1, outlet_marker, "Outlet")
    gmsh.model.addPhysicalGroup(1, obstacle, obstacle_marker)
    gmsh.model.setPhysicalName(1, obstacle_marker, "Obstacle")

  res_min = r / 4.0
  if mesh_comm.rank == model_rank:
    distance_field = gmsh.model.mesh.field.add("Distance")
    gmsh.model.mesh.field.setNumbers(distance_field, "EdgesList", obstacle)
    threshold_field = gmsh.model.mesh.field.add("Threshold")
    gmsh.model.mesh.field.setNumber(threshold_field, "IField", distance_field)
    gmsh.model.mesh.field.setNumber(threshold_field, "LcMin", res_min)
    gmsh.model.mesh.field.setNumber(threshold_field, "LcMax", 0.08 * H)
    gmsh.model.mesh.field.setNumber(threshold_field, "DistMin", r)
    gmsh.model.mesh.field.setNumber(threshold_field, "DistMax", 2 * H)
    min_field = gmsh.model.mesh.field.add("Min")
    gmsh.model.mesh.field.setNumbers(min_field, "FieldsList", [threshold_field])
    gmsh.model.mesh.field.setAsBackgroundMesh(min_field)

  if mesh_comm.rank == model_rank:
    gmsh.option.setNumber("Mesh.Algorithm", 6)
    gmsh.model.mesh.generate(gdim)

  msh, subdomains, boundaries = io.gmshio.model_to_mesh(gmsh.model, mesh_comm, model_rank, gdim=2)
  gmsh.finalize()

tdim = msh.topology.dim
num_cells = msh.topology.index_map(tdim).size_local
h = cpp.mesh.h(msh, tdim, range(num_cells))
hmin = h.min()

print(f"|> Mesh created with hmin={h.min():f}, hmax={h.max():f} and {num_cells:d} total elements")

|> Mesh created with hmin=0.010504, hmax=0.044935 and 3831 total elements


As usual, we use `pyvista` to visualize the mesh:

In [ ]:
def plot_mesh(mesh, cell_values=None, filename="file.html"):
    pyvista.start_xvfb()
    grid = pyvista.UnstructuredGrid(*plot.create_vtk_mesh(mesh))
    plotter = pyvista.Plotter(notebook=True, window_size=[500,500])

    if cell_values is not None:
        min_ = cell_values.x.array.min()
        max_ = cell_values.x.array.max()
        grid.cell_data["cell_values"] = cell_values.x.array
        viridis = plt.cm.get_cmap("viridis", 25)
        plotter.add_mesh(grid, cmap=viridis, show_edges=True, clim=[min_, max_])
    else:
        plotter.add_mesh(grid, show_edges=True)
    
    plotter.camera.zoom(2.0)
    plotter.view_xy()
    plotter.export_html(filename, backend="pythreejs")
    plotter.close()

with io.XDMFFile(MPI.COMM_WORLD, "mymesh.xdmf", "w") as xdmf:
    xdmf.write_mesh(msh)

plot_mesh(msh, cell_values=None, filename="mesh.html")
IPython.display.HTML(filename="mesh.html")

## Finite element spaces

Now, we reach a critical point in the implementation, which is the choice of the *discrete basis* (i.e., finite element spaces) for the **velocity** and **pressure** unknowns, which is not a trivial issue as in previous cases. 

In a constrained problem like this one, for the discrete formulation
to be well-posed, we need the velocity space $U_h \subset [H^1(\Omega)]^d$ and the pressure space $Q_h \subset L^2(\Omega)$ to satisfy a compatibility requirement, such that the famous LBB (**Ladyzhenskaya-Babuška-Brezzi**) condition is full-filled. Otherwise, some numerical artifacts known as
*spurious modes* will emerge. A complete explanation of such condition is out of the scope of this course. Another alternative is to adopt 
a stabilized formulation, such as GLS, ASGS, VMS, etc.m but this is left for a more advanced course.
$~$

One possible choice of a stable finite element pair is the *mini*-element 
($P_1^+$-$P_1$). One very popular and simple choice is the **Taylor-Hood** element. Given a finite element conforming mesh $\mathcal{T}_h$ we choose

$$
U(\mathcal{T}_h) = U_h = \{\mathbf{v} \in [H^1(\Omega)]^d,~\mathbf{v}|_E \in [P_2(E)]^d \, \forall E \in \mathcal{T}_h\}
$$

$$
Q(\mathcal{T}_h) = Q_h = \{\mathbf{v} \in H^1(\Omega),~\mathbf{v}|_E \in P_1(E) \, \forall E \in \mathcal{T}_h\}
$$

which is implemented in `FEniCSx` as:

In [ ]:
# Create an element for each field
degree_u = 2
P2 = VectorElement("CG", msh.ufl_cell(), degree_u)

degree_p = 1
P1 = FiniteElement("CG",  msh.ufl_cell(), degree_p)

# the Taylor-Hood element 
TH  = MixedElement([P2, P1])

# and the corresponding global space of functions
W = fem.FunctionSpace(msh, TH)

U, Umap = W.sub(0).collapse()
Q, Qmap = W.sub(1).collapse()

For the $P_2$ Lagrangian finite element space associated to a partition $\mathcal{T}_h$, the degrees of freedom (dofs) are the function values at:

* Vertices of the triangular elements
* Mid-points of edges

A discrete function $\mathbf{u}_h\in U_h$ is written as

$$
\mathbf{u}_h = \sum_{i=1}^{dim{U_h}}{a_j \mathbf{N}_j}
$$

where $\{\mathbf{N}_j\}_{j=1}^{dim{U_h}}$ are a set of basis functions for $U_h$. For the $P_1$ Lagrangian space, the dofs are simply the function values 
at the vertices of the triangulation, a discrete function 
$\mathbf{p}_h\in Q_h$ is written as

$$
p_h = \sum_{i=1}^{dim{Q_h}}{p_j \psi_j}
$$
where $\{\psi_j\}_{j=1}^{dim{U_h}}$ are a set of basis functions for $Q_h$. 

Therefore, the total dimension of the system to be solved is

$$
dim = dim{U_h} + dim{Q_h}
$$

so the global vector of unknowns $\boldsymbol{\mathsf{X}} = [\boldsymbol{\mathsf{U}}, \boldsymbol{\mathsf{P}}]^{\intercal} \in \mathbb{R}^{dim}$, 
where $\boldsymbol{\mathsf{U}} \in \mathbb{R}^{dimU_h}$ and $\boldsymbol{\mathsf{P}} \in \mathbb{R}^{dimP_h}$.



In [ ]:
# Create and auxliary function
faux = fem.Function(W)
dimtot = faux.x.array.shape[0]
dimU = faux.sub(0).collapse().x.array.shape[0]
dimP = faux.sub(1).collapse().x.array.shape[0]
print(f"|> Triangular mesh created having {num_cells:d} elements.\n|> #dofs = dimTotal = {dimtot:d} = dimU + dimP = {dimU:d} + {dimP:d}")

|> Triangular mesh created having 3831 elements.
|> #dofs = dimTotal = 17772 = dimU + dimP = 15750 + 2022


## Boundary conditions

Implementation of the Dirichlet or essential conditions are also a critical issue to as the discretization. 

For both problems to be solved, namely, the Poiseuille and the obtacle in a channel, we impose **non-slip** Dirichlet conditions for velocity at the *Top* and *Bottom* walls and an inlet velocity profile at the *left* wall, which is given by
$~$

$$
\mathbf{u}_{inlet}(x=0,y) = 2 U_{\max} H (H - y) / H^2
$$

where $H$ is the inlet height. The outlet (*right* wall) is left open,
so as the fluid is free to leave (stress-free).

In [ ]:
# Boundary values
Umax = 1
def u_inlet(x):
    return (2*Umax*x[1]*(Ly-x[1])/Ly**2, np.zeros_like(x[0]))

def u_noslip(x):
    return (np.zeros_like(x[0]), np.zeros_like(x[0]))

if(simple_mesh):

  def walls(x):
    return np.logical_or(np.isclose(x[1],0), np.isclose(x[1],Ly))

  def inflow(x):
    return np.isclose(x[0], 0)

  def outflow(x):
    return np.isclose(x[0], Lx)

  #------------
  wall_dofs = fem.locate_dofs_geometrical((W.sub(0), U), walls)
  u_ns = fem.Function(U)
  u_ns.interpolate(u_noslip)
  bc_noslip = fem.dirichletbc(u_ns, wall_dofs, W.sub(0))

  #------------
  inflow_dofs = fem.locate_dofs_geometrical((W.sub(0), U), inflow)
  u_in = fem.Function(U)
  u_in.interpolate(u_inlet)
  bc_inflow = fem.dirichletbc(u_in, inflow_dofs, W.sub(0))

  # Collect all bcs
  bcs = [bc_noslip, bc_inflow]
else:
  fdim = 1

  # Inlet
  inflow_dofs = fem.locate_dofs_topological((W.sub(0), U), fdim, boundaries.find(inlet_marker))
  u_in = fem.Function(U)
  u_in.interpolate(u_inlet)
  bc_inflow = fem.dirichletbc(u_in, inflow_dofs, W.sub(0))

  # Noslip walls
  wall_dofs = fem.locate_dofs_topological((W.sub(0), U), fdim, boundaries.find(wall_marker))
  obs_dofs = fem.locate_dofs_topological((W.sub(0), U), fdim, boundaries.find(obstacle_marker))
  u_ns = fem.Function(U)
  u_ns.interpolate(lambda x: u_noslip(x))
  bc_walls = fem.dirichletbc(u_ns, wall_dofs, W.sub(0))
  bc_obstacle = fem.dirichletbc(u_ns, obs_dofs, W.sub(0))
  
  # Collect all bcs
  bcs = [bc_inflow, bc_obstacle, bc_walls]

## Fully discrete problem

We finally provide the time discretization for this problem.
Ley us begin by discretizing the time derivative as 

$$
\left. \dfrac{\partial{\mathbf{u}}}{\partial{t}}\right |_{[t_n,t_{n+1}]} \approx \dfrac{\mathbf{u}^{n+1} - \mathbf{u}^n}{\Delta{t}}
$$

Now, we must choose at which time each term is 

\begin{eqnarray}
\int_{\Omega}{\rho\left (\dfrac{\mathbf{u}^{n+1} - \mathbf{u}^n}{\Delta{t}} +  \mathbf{u}^{n+\gamma_1} \cdot \nabla \mathbf{u}^{n+\gamma_2}\right ) \cdot \mathbf{v} }\,dx +
{\int_{\Omega}}{2\,\mu \boldsymbol{\varepsilon}(\mathbf{u}^{n+\theta}) : \boldsymbol{\varepsilon}(\mathbf{v}) }\,dx 
& = & \\
- {\int_{\Omega}}{p^{n+1}\,\nabla \cdot \mathbf{v}}\,dx = {\int_{\Omega}}{\mathbf{f}^{n+\theta}\cdot \mathbf{v}}\,dx + {\int_{\Gamma_{\boldsymbol{\mathcal{F}}}}}{\boldsymbol{\mathcal{F}^{n+\theta}} \cdot \mathbf{v}}\,ds \\
& &  \\
{\int_{\Omega}}{q\,\nabla \cdot \mathbf{u}^{n+1}}\,dx  & = & 0  
\end{eqnarray}

The algebraic problem would look like

$$
\begin{bmatrix}
A_{\mathbf{u}\mathbf{u}} & B_{\mathbf{u}p} \\
C_{p\mathbf{u}} & 0 
\end{bmatrix} 
\begin{bmatrix}
\mathsf{U}^{n+1} \\
\mathsf{P}^{n+1}
\end{bmatrix} 
= 
\begin{bmatrix}
\mathsf{F} \\
\mathsf{0}
\end{bmatrix} 
$$

where the rhs contains contributions from the forcing terms as
well as from the previous solution $\mathsf{U}^{n}$ and the boundary
conditions.

---

Some comments are in order:

* The pressure and continuity equation and always treated implicitly

* Depending on the choice for $\gamma_1$ and $\gamma_2$ we may end up with a linearized problem or a nonlinear one, for which, Newton-Raphson, Picard or fixed-point iterations must be applied. Below, we create a Newton solver using 
the two functions

      problem = fem.petsc.NonlinearProblem(...)
      solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem)

(*Notice that a nonlinear solver serves both, linear and nonlinear problems, except that in the former case it should converge in 1 single iteration)*

* Depending on the choice for $\theta$ we end up with an implicit or explicit formulation.
The latter case requiring the time step to be bounded by 

$$
\Delta{t}_{\mu} < C_{\mu} \dfrac{\rho \, h^2}{\mu}
$$
for some constant $C_{\mu}$. For the implicit case we
usually take $\theta = 1$ or $\theta = 0.5$ (Crank-Nicolson).

* In a totally explicit treatment of advective terms, advection is stabilized by viscosity if the time step is bounded by

$$
\Delta{t}_{a} < \dfrac{C_a\mu}{\rho \, (\mathbf{u} \cdot \mathbf{u})}
$$

* If we consider the Reynold number Re$=\dfrac{\rho \, U \, H}{\mu}$ and
nondimensionalize time as $\bar{t} = \dfrac{t U}{H}$, we see that
$$
\bar{\Delta{t}}_{a} = \dfrac{C_a}{\mbox{Re}} \xrightarrow[\mbox{Re} ~\rightarrow~ \infty]{} 0~~~~~\mbox{and}~~~~~
\bar{\Delta{t}}_{\mu} = C_{\mu} \dfrac{h^2}{H^2}\mbox{Re}\xrightarrow[\mbox{Re} ~\rightarrow~ 0]{} 0,
$$

* For low Re, its is convenient an implicit scheme for the viscous terms, 
since the restriction scales as $\mathcal{O}(h^2)$.

* For high Re, the time step should also be controlled, however, stability also
depends on the choice of the advection scheme.


---

In [ ]:
# Define numerical and material parameters

# Time-stepping parameters
T = 10
num_steps = 1000
dt = T/num_steps
Dt = fem.Constant(msh, ScalarType(dt))

# Time parameter for viscous term
theta = fem.Constant(msh, ScalarType(0.5))

# Time parameter for convective velocity
gamma = fem.Constant(msh, ScalarType(0.0))

# Body force
f = fem.Constant(msh, ScalarType((0,0)))

# Viscosity
muf = 0.001
mu = fem.Constant(msh, ScalarType(muf))

# Density
rhof = 3.0
rho = fem.Constant(msh, ScalarType(rhof))

print("-------------------------------")
print("--- To have as a reference ")
print(f"|> Time step: dt = {dt:f}")
print(f"|> dt_adv = {4*muf/(rhof*Umax**2):f}")
print(f"|> dt_vis = {0.25*rhof*hmin**2/muf:f}")
print(f"|> dt_cfl = {hmin/Umax:f}")
print("-------------------------------")

if(simple_mesh):
  Re = Ly*rhof*Umax/muf
else:
  Re = 2*r*Umax*rhof/muf

print("-------------------------------")
print(f"|> Reynolds number: Re = {Re:0.1f}")
print("-------------------------------")

-------------------------------
--- To have as a reference 
|> Time step: dt = 0.010000
|> dt_adv = 0.001333
|> dt_vis = 0.082758
|> dt_cfl = 0.010504
-------------------------------
-------------------------------
|> Reynolds number: Re = 300.0
-------------------------------


In [ ]:
# Strain-rate tensor
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

# Current solution
w = fem.Function(W)

# Solution at previous time
w_n = fem.Function(W)

dw = TestFunction(W)

(u, p) = split(w)
(v, q) = split(dw)

w_n.sub(0).interpolate(u_inlet)
(u_n, p_n) = split(w_n) # é o split do uf

utheta = theta*u + (1-theta)*u_n
ugamma = gamma*u + (1-gamma)*u_n

# Momentum residual
Fu  = rho*inner((u - u_n) / Dt, v) * dx
Fu += rho*inner(dot(ugamma, nabla_grad(utheta)), v) * dx
Fu += inner(2*mu*epsilon(utheta), epsilon(v)) * dx - p * div(v) * dx
Fu -= inner(f, v)*dx
# Continuity residual
Fp = div(u) * q * dx
# Total residual
Res = Fu + Fp

# Setting the Non-linear solver
Jac = derivative(Res, w)
problem = fem.petsc.NonlinearProblem(Res, u=w, bcs=bcs, J=Jac)
solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6
solver.max_it = 10
solver.report = True

# Setting linear solver options to be used at each Newton iteration
ksp = solver.krylov_solver
opts = Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
opts[f"{option_prefix}ksp_monitor"] = None
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()


In [ ]:
# Loop over time steps
t = 0

# Save initial condition
xdmfu = io.XDMFFile(msh.comm, "vel.xdmf", "w")
xdmfu.write_mesh(msh)
upl = w_n.sub(0).collapse()
upl.name = "Velocity"
xdmfu.write_function(upl, t)

xdmfp = io.XDMFFile(msh.comm, "pres.xdmf", "w")
xdmfp.write_mesh(msh)
ppl = w_n.sub(1).collapse()
ppl.name = "Pressure"
xdmfp.write_function(ppl, t)

freq_out = 50
for i in range(num_steps):
    
    t += dt

    print("-------------------------------")
    print(f"|> Solving at time= {t:f}\n")
    log.set_log_level(log.LogLevel.INFO)
    niters, converged = solver.solve(w)
    assert(converged)
    print(f"Number of interations= {niters:d}")

    # Update the previous solution with the last computed value
    w_n.x.array[:] = w.x.array[:]
    w_n.x.scatter_forward()

    # Save the last computed solution
    if(i % freq_out== 0):
      upl = w_n.sub(0).collapse()
      upl.name = "Velocity"
      xdmfu.write_function(upl, t)
      ppl = w_n.sub(1).collapse()
      ppl.name = "Pressure"
      xdmfp.write_function(ppl, t)

# End loop over time steps
xdmfu.close()
xdmfp.close()

from google.colab import files
files.download('vel.xdmf')
files.download('vel.h5')
files.download('pres.xdmf')
files.download('pres.h5')

A saída de streaming foi truncada nas últimas 5000 linhas.
-------------------------------
|> Solving at time= 5.010000

  Residual norms for nls_solve_ solve.
  0 KSP Residual norm 2.448517947115e-02 
  1 KSP Residual norm 4.554546009937e-18 
  Residual norms for nls_solve_ solve.
  0 KSP Residual norm 8.159474544325e-17 
  1 KSP Residual norm 3.974897064232e-31 
Number of interations= 2
-------------------------------
|> Solving at time= 5.020000

  Residual norms for nls_solve_ solve.
  0 KSP Residual norm 2.448678153356e-02 
  1 KSP Residual norm 4.699836972514e-18 
  Residual norms for nls_solve_ solve.
  0 KSP Residual norm 8.354624119976e-17 
  1 KSP Residual norm 2.633552737254e-30 
Number of interations= 2
-------------------------------
|> Solving at time= 5.030000

  Residual norms for nls_solve_ solve.
  0 KSP Residual norm 2.449281884698e-02 
  1 KSP Residual norm 5.002118469364e-18 
  Residual norms for nls_solve_ solve.
  0 KSP Residual norm 8.324591427627e-17 
  1 KSP R

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Homework 4

1. **Computing forces on the object**

Consider the computation of the forces over the obtacle that are given by

$$
\mathbf{F} = \int_{\Gamma_{obs}}{\boldsymbol{\sigma}(\mathbf{u},p) \cdot \check{\mathbf{n}}}\,ds = 
$$

where 

$$
\boldsymbol{\sigma}(\mathbf{u},p) = -p\mathbf{I} + 2\,\mu\, \boldsymbol{\varepsilon}(\mathbf{u}) 
$$

The task for this homerwork is:

* Check the implementation in the cell below and make sure to understand the details.

* Take `T = 10` and `num_steps = 1500`, $\gamma = 0$, $\theta = 0.5$. Run the code for three different mesh refinements to compare the results of $\mathbf{F}$ as a function of time. In order to change the mesh refinement got the cell where the mesh is created and set the variable `res_min` to an appropriate
value, e.g. `r/4`, `r/8` and `r/16`. The latter case may take some time to 
run in the colab environment.

In [ ]:
n = -FacetNormal(msh) # Normal pointing out of obstacle
ds_obs = Measure("ds", domain=msh, subdomain_data=boundaries, subdomain_id=obstacle_marker)

def sigma(u,p):
  return 2*mu*epsilon(u) - p*Identity(2)

# Loop over time steps
t = 0
teval = np.linspace(0,T, num_steps, dtype=np.float64)
forcex = np.zeros(num_steps, dtype=np.float64)
forcey = np.zeros(num_steps, dtype=np.float64)
for i in range(num_steps):
    
    t += dt

    print("-------------------------------")
    print(f"|> Solving at time= {t:f}\n")
    log.set_log_level(log.LogLevel.INFO)
    niters, converged = solver.solve(w)
    assert(converged)
    print(f"Number of interations= {niters:d}")

    # Update the previous solution with the last computed value
    w_n.x.array[:] = w.x.array[:]
    w_n.x.scatter_forward()

    upl = w_n.sub(0).collapse()
    ppl = w_n.sub(1).collapse()
    
    forcexform = fem.form( dot(sigma(upl,ppl), n)[0] * ds_obs )
    forceyform = fem.form( dot(sigma(upl,ppl), n)[1] * ds_obs )
    forcex[i] = msh.comm.allreduce(fem.assemble_scalar(forcexform), op=MPI.SUM)
    forcey[i] = msh.comm.allreduce(fem.assemble_scalar(forceyform), op=MPI.SUM)

# End loop over time steps

In [ ]:
plt.figure(figsize= (9,6))
plt.rcParams['font.size'] = '16'
plt.axes(facecolor = "lightgray")
plt.xlabel('time',fontsize=20)
plt.ylabel('Force', fontsize=20)
l = plt.plot(teval[10:], forcex[10:], '-r', teval[10:], forcey[10:], '-b')
plt.setp(l, 'markersize', 6)
plt.setp(l, 'markerfacecolor', 'white')
plt.legend(['$F_x(t) = \int_{\Gamma_{obs}}\mathcal{F}_x\,ds$'])
plt.grid()
plt.show()

2. **OPTIONAL: Stabilization of convective terms**

The treatment of the convective term 

$$
(\nabla\mathbf{u})\cdot \mathbf{u}= (\mathbf{u} \cdot \nabla)(\mathbf{u})
$$ 

is also a delicate issue. A robust implementation would include some sort of stabilization, tipically, the **SUPG** (Streamline Upwind Petrov Galerkin) method, which amounts to add the following term to the variational formulation

\begin{equation}
\mbox{SUPG} = \sum_{K \in \mathcal{T}_h}{\tau_K \,\int_{K}{\mathscr{P}(\mathbf{v}_h) \cdot \mathscr{R}(\mathbf{u}_h)}}\,dx \nonumber
\end{equation}

where 
$\mathscr{R}(\mathbf{u}) = \rho\left (\dfrac{\partial{\mathbf{u}}}{\partial{t}} +  \mathbf{u} \cdot \nabla \mathbf{u}\right )
- \nabla \cdot ( 2 \, \mu \boldsymbol{\varepsilon}(\mathbf{u})) + \nabla{p} - \mathbf{f}$,$~$ is the residual of the momentum equation
and the elementwise stabilization parameter $\tau_K$ is taken as

\begin{equation}
\tau_K = \left [ c_1\frac{\mu}{\rho \, h_K^2} + c_2\frac{\|\mathbf{u}\|_{\infty,K}}{h_K} \right ]^{-1} \nonumber
\end{equation}

and the perturbed test function $\mathscr{P}(\mathbf{v})$ is 

\begin{equation}
\mathscr{P}(\mathbf{v}) = (\mathbf{u} \cdot \nabla) \mathbf{v}  \nonumber
\end{equation}

For our current implementation and for the problems to be solved we will assume the convective term is controlled by limiting the Reynolds number Re$=\dfrac{\rho \, U_{\max} H}{\mu}$ to relatively low values.

*Implementation of such stabilization method is left as an optional exercise for the advanced students*.
